<a href="https://colab.research.google.com/github/Barclaysdev2011/data_science/blob/main/churnPredictionfinalProject_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3 pandas openpyxl

In [ ]:
import boto3
import pandas as pd
from io import BytesIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter, CategoricalParameter
from sagemaker.xgboost.estimator import XGBoost

# Initialize the S3 client using the default SageMaker role
s3 = boto3.client('s3')

# Define the S3 bucket and file key
bucket_name = 'sagemaker-us-east-2-971422683094'
file_key = 'storedata_total.xlsx'

# Fetch the Excel file from S3
response = s3.get_object(Bucket=bucket_name, Key=file_key)

# Read the file content into a pandas DataFrame
excel_data = response['Body'].read()
data = pd.read_excel(BytesIO(excel_data))

# Display the DataFrame
data.head()  # Preview the first few rows of the DataFrame


/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,custid,retained,created,firstorder,lastorder,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,favday,city
0,6H6T6N,0,2012-09-28,2013-08-11 00:00:00,2013-08-11 00:00:00,29,100.000000,3.448276,14.52,0.000000,0,0,0,Monday,DEL
1,APCENR,1,2010-12-19,2011-04-01 00:00:00,2014-01-19 00:00:00,95,92.631579,10.526316,83.69,0.181641,1,1,1,Friday,DEL
2,7UP6MS,0,2010-10-03,2010-12-01 00:00:00,2011-07-06 00:00:00,0,0.000000,0.000000,33.58,0.059908,0,0,0,Wednesday,DEL
3,7ZEW8G,0,2010-10-22,2011-03-28 00:00:00,2011-03-28 00:00:00,0,0.000000,0.000000,54.96,0.000000,0,0,0,Thursday,BOM
4,8V726M,1,2010-11-27,2010-11-29 00:00:00,2013-01-28 00:00:00,30,90.000000,13.333333,111.91,0.008850,0,0,0,Monday,BOM


In [ ]:
# Handle missing values by dropping rows with missing `custid` or `created`
data = data.dropna(subset=['custid', 'created'])

In [ ]:
# Convert date columns to datetime type
data['firstorder'] = pd.to_datetime(data['firstorder'], errors='coerce')
data['lastorder'] = pd.to_datetime(data['lastorder'], errors='coerce')
data.head()

,custid,retained,created,firstorder,lastorder,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,favday,city
0,6H6T6N,0,2012-09-28,2013-08-11,2013-08-11,29,100.000000,3.448276,14.52,0.000000,0,0,0,Monday,DEL
1,APCENR,1,2010-12-19,2011-04-01,2014-01-19,95,92.631579,10.526316,83.69,0.181641,1,1,1,Friday,DEL
2,7UP6MS,0,2010-10-03,2010-12-01,2011-07-06,0,0.000000,0.000000,33.58,0.059908,0,0,0,Wednesday,DEL
3,7ZEW8G,0,2010-10-22,2011-03-28,2011-03-28,0,0.000000,0.000000,54.96,0.000000,0,0,0,Thursday,BOM
4,8V726M,1,2010-11-27,2010-11-29,2013-01-28,30,90.000000,13.333333,111.91,0.008850,0,0,0,Monday,BOM


In [ ]:
# Encode categorical columns (`favday`, `city`)
label_encoder_favday = LabelEncoder()
label_encoder_city = LabelEncoder()
data['favday'] = label_encoder_favday.fit_transform(data['favday'])
data['city'] = label_encoder_city.fit_transform(data['city'])

In [ ]:
# Prepare the target and feature sets
X = data.drop(columns=['custid', 'retained'])
y = data['retained']

In [ ]:
# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()
print(sagemaker_session.default_bucket())

sagemaker-us-east-2-971422683094


In [ ]:
# Set up the XGBoost estimator
xgb = XGBoost(
    entry_point='train.py',
    framework_version='1.5-1',
    instance_type='ml.m5.xlarge',
    instance_count=1,
    output_path=f's3://{sagemaker_session.default_bucket()}/output',
    sagemaker_session=sagemaker_session,
    role=role,
)

In [ ]:
hyperparameter_ranges = {
    'max_depth': IntegerParameter(3, 10),
    'eta': ContinuousParameter(0.01, 0.3),
    'gamma': ContinuousParameter(0, 5),
    'min_child_weight': IntegerParameter(1, 10),
    'subsample': ContinuousParameter(0.5, 1.0),
}

In [ ]:
tuner = HyperparameterTuner(
    xgb,
    objective_metric_name='validation:auc',
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=20,
    max_parallel_jobs=3,
)

In [ ]:
tuner.fit({'train': 's3://arn:aws:iam::971422683094:role/service-role/AmazonSageMaker-ExecutionRole-20241113T114041', 'validation': 's3://arn:aws:iam::971422683094:role/service-role/AmazonSageMaker-ExecutionRole-20241113T114041'})

In [ ]:
from sagemaker.estimator import Estimator

# Attach the best model from tuning
best_model = tuner.best_estimator()

# Deploy or evaluate the model (download model artifacts and run predictions)
best_model.download('best_model.tar.gz')
# Unzip and load the model locally if needed for evaluation

# Evaluate AUC on the test set
from sklearn.metrics import roc_auc_score

y_pred = best_model.predict(X_test)
auc_score = roc_auc_score(y_test, y_pred)
print(f'AUC Score: {auc_score}')

# Check if the AUC is above a threshold (e.g., 0.75)
if auc_score > 0.75:
    print("Proceeding to register the model.")


In [ ]:
# Register the model
model_package = best_model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="CustomerChurnModelGroup",
    approval_status="Approved",
)


In [ ]:
# Create a model for deployment
sagemaker_model = best_model.create_model(
    instance_type='ml.m5.large',
    role=role,
)


In [ ]:
transformer = sagemaker_model.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{sagemaker_session.default_bucket()}/batch-transform-output',
)

# Run batch transform
transformer.transform(data='s3://path-to-batch-input', content_type='text/csv', split_type='Line')
transformer.wait()


In [ ]:
clarify_config = {
    "bias_config": {
        "label_values_or_threshold": [1],
        "facet_name": "city",
        "facet_values_or_threshold": ["DEL"],
    },
    "shap_config": {
        "baseline": X_train.iloc[:100].mean().tolist(),  # Example baseline
        "num_samples": 100,
    },
    "predictor_config": {
        "instance_type": "ml.m5.large",
        "instance_count": 1,
        "initial_instance_count": 1,
    },
}


